In [ ]:

import pandas as pd
import numpy as np
import re
from datetime import datetime

class FertilizerStandardizer:
    def __init__(self):
        self.icv_fertilizer_list = ["Solution Liquide N 39"]
        self.default_mineral_density = 1.325
        self.organic_solid_density = 1000
        self.organic_liquid_density = 1.0
        self.type_mapping = {
            "Fertilisation et amendement organique": "engrais organiques",
            "Fertilisation et amendement mineral - foliaire inclus": "engrais mineraux",
            "Fertilisation organique": "engrais organiques",
            "Fertilisation minérale": "engrais mineraux"
        }

    def standardize_fertilizer_name(self, name):
        if pd.isna(name): return name
        if isinstance(name, (pd.Timestamp, datetime)):
            name = name.strftime('%d-%m-%Y')
        name = str(name).upper()
        name = re.sub(r'[^\w\s\-]', ' ', name)
        name = re.sub(r'\s+', ' ', name)
        return name.strip()

    def is_in_icv_list(self, fertilizer_name):
        std_name = self.standardize_fertilizer_name(fertilizer_name)
        std_icv_list = [self.standardize_fertilizer_name(name) for name in self.icv_fertilizer_list]
        return std_name in std_icv_list

    def standardize_type_engrais(self, intervention_type):
        if pd.isna(intervention_type):
            return "engrais mineraux"
        type_std = self.type_mapping.get(intervention_type)
        if type_std is None:
            print(f"Avertissement: Type d'intervention non reconnu: {intervention_type}")
            return "engrais mineraux"
        return type_std

    def convert_to_kg(self, row):
        try:
            if pd.isna(row.get('unite_intrant_intervention')) or pd.isna(row.get('quantite_totale')):
                return np.nan
            unite = str(row.get('unite_intrant_intervention', '')).upper()
            type_engrais = str(row.get('type_engrais', '')).lower()
            quantite = float(row.get('quantite_totale', 0))
            
            if unite == 'KG':
                return quantite
            elif unite == 'L':
                return quantite * (self.organic_liquid_density if 'organique' in type_engrais else self.default_mineral_density)
            elif unite == 'M3':
                if 'organique' in type_engrais:
                    return quantite * self.organic_solid_density
                return quantite * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row.get('libelle', 'Unknown')}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row.get('libelle', 'Unknown')}: {str(e)}")
            return np.nan

    def convert_dose_to_kg_ha(self, row):
        try:
            if pd.isna(row.get('unite')) or pd.isna(row.get('dose')):
                return np.nan
            
            unite = str(row['unite']).upper()
            if unite == 'KG/HA':
                return row['dose']
            elif unite == 'T/HA':
                return row['dose'] * 1000
            elif unite == 'L/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_liquid_density
                return row['dose'] * self.default_mineral_density
            elif unite == 'M3/HA':
                if row['type_engrais'] == 'engrais organiques':
                    return row['dose'] * self.organic_solid_density
                return row['dose'] * self.default_mineral_density * 1000
            else:
                print(f"Avertissement: Unité non reconnue: {unite} pour l'engrais {row['libelle']}")
                return np.nan
        except Exception as e:
            print(f"Erreur lors de la conversion pour l'engrais {row['libelle']}: {str(e)}")
            return np.nan

    def calculate_nutrients(self, row):
        try:
            if not row['in_icv_list']:
                if row['type_engrais'] == 'engrais organiques':
                    volume_m3 = row['quantite_totale'] if row['unite_intrant_intervention'].upper() == 'M3' \
                        else row['quantite_kg'] / self.organic_solid_density
                    
                    row['N_kg_ha'] = row['fertilisant.composition.n_total'] * volume_m3 / row['surface_travaillee_ha']
                    row['P_kg_ha'] = row['fertilisant.composition.p'] * volume_m3 / row['surface_travaillee_ha']
                    row['K_kg_ha'] = row['fertilisant.composition.k'] * volume_m3 / row['surface_travaillee_ha']
                else:
                    row['N_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.n_total'] / 100
                    row['P_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.p'] / 100
                    row['K_kg_ha'] = row['dose_kg_ha'] * row['fertilisant.composition.k'] / 100
            return row
        except Exception as e:
            print(f"Erreur lors du calcul des nutriments pour l'engrais {row['libelle']}: {str(e)}")
            return row

    def standardize_mesparcelles(self, data, aggregate=False):
        """Standardise les données MesParcelles"""
        print("Standardisation des données MesParcelles...")
        
        fertilisation_types = [
            "Fertilisation et amendement organique",
            "Fertilisation et amendement mineral - foliaire inclus"
        ]
        df = data[data['type_intervention.libelle'].isin(fertilisation_types)].copy()
        
        if 'culture.libelle' in df.columns:
            df = df.rename(columns={'culture.libelle': 'culture'})
            print("Colonne culture.libelle trouvée et renommée en 'culture'")
        else:
            print("ATTENTION: culture.libelle non trouvée!")
            print("Colonnes disponibles:", df.columns.tolist())
            return None
            
        df['type_engrais'] = df['type_intervention.libelle'].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        df['quantite_kg'] = df.apply(self.convert_to_kg, axis=1)
        df['dose_kg_ha'] = df['quantite_kg'] / df['surface_travaillee_ha']
        df = df.apply(self.calculate_nutrients, axis=1)
        
        if aggregate:
            return self._aggregate_by_exploitation_and_culture(df)
        return df

    def standardize_smag(self, data, cultures_data, aggregate=False):
        """Standardise les données SMAG"""
        print("\nStandardisation des données SMAG...")
        
        # Filtrer les fertilisations
        fertilisation_types = ["Fertilisation organique", "Fertilisation minérale"]
        df = data[data["Type d'intervention"].isin(fertilisation_types)].copy()
        
        # Renommage initial des colonnes
        base_columns = {
            'Intrant': 'libelle',
            'Unité': 'unite',
            'Dose': 'dose',
            'Surface intervention sur parcelle': 'surface',  # On utilise directement cette colonne pour la jointure
            'N': 'N_kg_ha',
            'P': 'P_kg_ha',
            'K': 'K_kg_ha',
            'SIRET': 'siret_exploitation',
            'Nom de la parcelle': 'uuid_parcelle'
        }
        df = df.rename(columns=base_columns)
        
        # Assurons-nous que les colonnes sont du même type dans les deux dataframes
        df['siret_exploitation'] = df['siret_exploitation'].astype(str)
        cultures_data['SIRET'] = cultures_data['SIRET'].astype(str)
        df['surface'] = df['surface'].astype(float)
        cultures_data['Surface'] = cultures_data['Surface'].astype(float)
        
        # Fusion avec cultures en utilisant SIRET, Nom de la parcelle ET Surface
        df = pd.merge(
            df,
            cultures_data[['SIRET', 'Nom de la parcelle', 'Surface', 'Culture']],
            left_on=['siret_exploitation', 'uuid_parcelle', 'surface'],
            right_on=['SIRET', 'Nom de la parcelle', 'Surface'],
            how='left'
        )
        
        # Nettoyage et renommage final
        df = df.rename(columns={'Culture': 'culture'})
        # On supprime les colonnes redondantes après la jointure
        df = df.drop(['SIRET', 'Nom de la parcelle', 'Surface'], axis=1, errors='ignore')
        
        # Standardisation
        df['type_engrais'] = df["Type d'intervention"].apply(self.standardize_type_engrais)
        df['in_icv_list'] = df['libelle'].apply(self.is_in_icv_list)
        df.loc[df['in_icv_list'], 'dose_kg_ha'] = df[df['in_icv_list']].apply(
            self.convert_dose_to_kg_ha, axis=1
        )
        
        # Log pour vérifier si la jointure a bien fonctionné
        total_rows = len(df)
        matched_rows = df['culture'].notna().sum()
        print(f"\nStatistiques de la jointure:")
        print(f"Nombre total de lignes: {total_rows}")
        print(f"Lignes avec une culture correspondante: {matched_rows}")
        print(f"Lignes sans correspondance: {total_rows - matched_rows}")
        
        return df if not aggregate else self._aggregate_by_exploitation_and_culture(df)

    def _aggregate_by_exploitation_and_culture(self, df):
        """Agrège les données par exploitation et par culture avec pondération par surface"""
        def weighted_average(group):
            result = pd.Series()
            
            # Conversion en string et liste des engrais
            group['libelle'] = group['libelle'].astype(str)
            result['engrais_utilises'] = ', '.join(sorted(group['libelle'].unique()))
            
            # Pour chaque engrais ICV, créer sa colonne de dose
            for icv_engrais in self.icv_fertilizer_list:
                icv_engrais_str = str(icv_engrais)
                mask = (group['in_icv_list']) & (group['libelle'].str.contains(icv_engrais_str, case=False, regex=False))
                if mask.any():
                    result[f"{icv_engrais_str}_dose"] = np.average(
                        group.loc[mask, 'dose_kg_ha'],
                        weights=group.loc[mask, 'surface_travaillee_ha']
                    )
                else:
                    result[f"{icv_engrais_str}_dose"] = np.nan
            
            # Pour les nutriments non-ICV
            non_icv_mask = ~group['in_icv_list']
            if non_icv_mask.any():
                for nutrient in ['N_kg_ha', 'P_kg_ha', 'K_kg_ha']:
                    if nutrient in group.columns:
                        values = group.loc[non_icv_mask, nutrient]
                        weights_subset = group.loc[non_icv_mask, 'surface_travaillee_ha']
                        if not values.empty and not weights_subset.empty:
                            result[f"non_icv_{nutrient}"] = np.average(
                                values,
                                weights=weights_subset
                            )
                        else:
                            result[f"non_icv_{nutrient}"] = np.nan
            else:
                for nutrient in ['N_kg_ha', 'P_kg_ha', 'K_kg_ha']:
                    result[f"non_icv_{nutrient}"] = np.nan
                    
            return result
        
        # Convertir la colonne libelle en string avant le groupby
        df['libelle'] = df['libelle'].astype(str)
        
        return df.groupby(['siret_exploitation', 'culture'], as_index=False).apply(weighted_average)
    def standardize_columns(self, df):
        """
        Transforme les données pour avoir une ligne par combinaison exploitation-parcelle-culture
        """
        print("Colonnes disponibles avant standardisation:", df.columns.tolist())
        
        
        # Convertir les colonnes en string pour éviter les problèmes de comparaison
        for col in ['libelle', 'siret_exploitation', 'uuid_parcelle', 'culture']:
            if col in df.columns:
                df[col] = df[col].astype(str)
        
        # Vérifier les colonnes nécessaires
        required_columns = ['siret_exploitation', 'uuid_parcelle', 'culture', 
                        'source', 'libelle', 'in_icv_list', 'dose_kg_ha',
                        'N_kg_ha', 'P_kg_ha', 'K_kg_ha']
                        
        for col in required_columns:
            if col not in df.columns:
                df[col] = np.nan
        
        # Gérer les valeurs manquantes avant le pivot
        df['libelle'] = df['libelle'].fillna('Non spécifié')
        
        try:
            # Pivot avec uuid_parcelle
            icv_doses = df[df['in_icv_list']].pivot_table(
                index=['siret_exploitation', 'uuid_parcelle', 'culture', 'source'],
                columns='libelle',
                values='dose_kg_ha',
                aggfunc='first'
            ).add_suffix('_dose')
            
            # Somme des nutriments
            non_icv_nutrients = df[~df['in_icv_list']].groupby(
                ['siret_exploitation', 'uuid_parcelle', 'culture', 'source']
            ).agg({
                'N_kg_ha': 'sum',
                'P_kg_ha': 'sum',
                'K_kg_ha': 'sum'
            }).rename(columns={
                'N_kg_ha': 'N_total',
                'P_kg_ha': 'P_total',
                'K_kg_ha': 'K_total'
            })
            
            # Combiner
            result = pd.merge(
                icv_doses,
                non_icv_nutrients,
                left_index=True,
                right_index=True,
                how='outer'
            ).reset_index()
            
            print("\nColonnes finales après transformation:")
            print(result.columns.tolist())
            
            return result
            
        except Exception as e:
            print(f"Erreur lors de la transformation: {str(e)}")
            print("État des données:")
            print(df.head())
            # En cas d'erreur, retourner les données brutes
            return df

    def combine_sources(self, mesparcelles_data, smag_data, smag_cultures_data):
        print("\nTraitement des données MesParcelles...")
        mp_std = self.standardize_mesparcelles(mesparcelles_data, aggregate=False)
        
        print("\nTraitement des données SMAG...")
        smag_std = self.standardize_smag(smag_data, smag_cultures_data, aggregate=False)
        
        if mp_std is None or smag_std is None:
            print("ERREUR: problème dans la standardisation des données")
            return None, None
        
        # Ajout de la source avant la standardisation des colonnes
        mp_std['source'] = 'mesparcelles'
        smag_std['source'] = 'smag'
        
        # Combiner avant de standardiser les colonnes
        combined_raw = pd.concat([mp_std, smag_std], ignore_index=True)
        
        # Standardiser les colonnes des données combinées
        standardized_raw = self.standardize_columns(combined_raw)
        
        # Faire l'agrégation sur les données combinées
        combined_aggregated = self._aggregate_by_exploitation_and_culture(combined_raw.copy())
        
        return standardized_raw, combined_aggregated

# Code d'utilisation
if __name__ == "__main__":
    print("\nImport des données...")
    
    mp_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\mesparcelles_data.xlsx"
    smag_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG.xlsx"
    smag_cultures_path = r"C:\Users\MonirNajem\OneDrive - FOOD PILOT\Desktop\monir\MESPARCELLES\SMAG_cultures.xlsx"
    
    mp_data = pd.read_excel(mp_path, sheet_name='Intervention_Details')
    smag_data = pd.read_excel(smag_path)
    smag_cultures = pd.read_excel(smag_cultures_path)
    
    standardizer = FertilizerStandardizer()
    raw_data, aggregated_data = standardizer.combine_sources(mp_data, smag_data, smag_cultures)
    
    if raw_data is not None and aggregated_data is not None:
        # Export dans un seul fichier Excel avec deux feuilles
        output_path = 'resultats_standardisation.xlsx'
        with pd.ExcelWriter(output_path) as writer:
            raw_data.to_excel(writer, sheet_name='Données brutes', index=False)
            aggregated_data.to_excel(writer, sheet_name='Données agrégées', index=False)
        print(f"\nRésultats exportés dans '{output_path}'")
        
        # Statistiques
        print("\nStatistiques données brutes:")
        print(f"Nombre total de parcelles: {len(raw_data)}")
        print(f"Nombre d'exploitations: {raw_data['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {raw_data['culture'].nunique()}")
        
        print("\nStatistiques données agrégées:")
        print(f"Nombre de combinaisons exploitation-culture: {len(aggregated_data)}")
        print(f"Nombre d'exploitations: {aggregated_data['siret_exploitation'].nunique()}")
        print(f"Nombre de cultures: {aggregated_data['culture'].nunique()}")



Import des données...

Traitement des données MesParcelles...
Standardisation des données MesParcelles...
Colonne culture.libelle trouvée et renommée en 'culture'

Traitement des données SMAG...

Standardisation des données SMAG...

Statistiques de la jointure:
Nombre total de lignes: 19776
Lignes avec une culture correspondante: 18981
Lignes sans correspondance: 795
Colonnes disponibles avant standardisation: ['K_kg_ha', 'N_kg_ha', 'P_kg_ha', 'culture', 'culture.id_culture', 'date_debut', 'date_fin', 'dose_kg_ha', 'fertilisant', 'fertilisant.composition.cao', 'fertilisant.composition.k', 'fertilisant.composition.mgo', 'fertilisant.composition.n_total', 'fertilisant.composition.p', 'fertilisant.composition.s', 'fertilisant.composition.unite', 'fertilisant.condition_epandage', 'fertilisant.condition_epandage.id_condition_epandage', 'fertilisant.condition_epandage.libelle', 'id_intrant', 'in_icv_list', 'libelle', 'link_parcelle', 'materiels', 'numero_lot', 'phyto', 'phyto.cible', 'phyto

C:\Users\MonirNajem\AppData\Local\Temp\ipykernel_14436\410655913.py:240: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(['siret_exploitation', 'culture'], as_index=False).apply(weighted_average)



Résultats exportés dans 'resultats_standardisation.xlsx'

Statistiques données brutes:
Nombre total de parcelles: 3539
Nombre d'exploitations: 202
Nombre de cultures: 59

Statistiques données agrégées:
Nombre de combinaisons exploitation-culture: 690
Nombre d'exploitations: 202
Nombre de cultures: 59
